# Genomic BPE Tests

In [ ]:
import os
import torch

# sets device for model and PyTorch tensors
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7" # "-1" if want to run on a CPU, otherwise define the GPU number

print('number of devices: {}'.format(torch.cuda.device_count()))

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
from pathlib import Path
import collections

from Bio import SeqIO
from fastai.text import *

import sentencepiece as spm

In [ ]:
#arbitrary number chosen. Original true "max sentence length" would be 44674217
max_sentence_len = 1100  #max size is 20480 

In [ ]:
#path = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes')
#path_to_files = path / 'genome_fastas'
path = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/ecoli/')
path_to_files = path / 'just_fastas'
path_to_all_files_txt = path / 'sentencepiece_tokenizer/all_files.txt'

In [ ]:
for i, file in enumerate(path_to_files.iterdir()):
    genome = SeqIO.parse(file, 'fasta')
    chroms = [GB for GB in genome if ('chromosome' in GB.description) or ('complete genome' in GB.description)] #  remove plasmid
    genome = ''.join([i.seq.__str__() for i in chroms]).upper()
    genome += '\n'
    with open(path_to_all_files_txt, 'a') as f:
        ind = 0
        while ind < len(genome):
            f.write(genome[ind:ind+max_sentence_len])
            f.write('\n')
            ind += max_sentence_len
    print('completed file {}/{}'.format(i,len(path_to_files.ls())))

In [ ]:
quotemark = '\"'
raw_text_path = path_to_all_files_txt
lang = 'dna'
pre_rules=None
post_rules=None
vocab_sz=None
max_vocab_sz = 10000
vocab_size = 100
model_type='bpe'
char_coverage=None
tmp_dir='tmp' 
enc='utf8'
coverage = 0.9998
spec_tokens = ['\u2581'+s for s in defaults.text_spec_tok]
spec_tokens.extend([str(i) for i in range(1, 21)])
cache_dir = path / 'sentencepiece_tokenizer' 
cache_dir = cache_dir / tmp_dir

#changed line in fastai.text see: https://forums.fast.ai/t/multifit-runtime-error-permission-denied/72874/3
# in the end not creating model thru fastai. using sentencepiece package directly!
print(" ".join([
        f"--input={quotemark}{raw_text_path}{quotemark} --max_sentence_length={max_sentence_len}",
        f"--character_coverage={coverage}",
        f"--unk_id={len(defaults.text_spec_tok)} --pad_id=-1 --bos_id=-1 --eos_id=-1",
        f"--user_defined_symbols={','.join(spec_tokens)}",
        f"--model_prefix={cache_dir/'spm'} --vocab_size={vocab_size} --model_type={model_type}"]))

spm.SentencePieceTrainer.Train(" ".join([
        f"--input={quotemark}{raw_text_path}{quotemark} --max_sentence_length={max_sentence_len}",
        f"--character_coverage={coverage}",
        f"--unk_id={len(defaults.text_spec_tok)} --pad_id=-1 --bos_id=-1 --eos_id=-1",
        f"--user_defined_symbols={','.join(spec_tokens)}",
        f"--model_prefix={cache_dir/'spm'} --vocab_size={vocab_size} --model_type={model_type}"]))

In [ ]:
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes/sentencepiece_tokenizer/tmp/spm.model')

In [ ]:
vocabs = [[s.id_to_piece(id), id] for id in range(s.get_piece_size())]

In [ ]:
vocabs[0:31]

In [ ]:
import random
test_text = "".join([random.choice("ATCG") for _ in range(1000)])

print(test_text)
print(s.encode(test_text, out_type=str, enable_sampling=True, alpha=0.1)[0:10])
print(s.encode(test_text, out_type=int, enable_sampling=True, alpha=0.1)[0:10])

test_text_2 = "1 2 3 4"

print(test_text_2)
print(s.encode(test_text_2, out_type=str, enable_sampling=True, alpha=0.1)[0:10])
print(s.encode(test_text_2, out_type=int, enable_sampling=True, alpha=0.1)[0:10])


#  Trying out model

## Download Sentencepiece model

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from typing import Callable
import torch

In [ ]:
from fastai import *
from fastai.text import ( BaseTokenizer, Tokenizer, Vocab, 
                         PreProcessor, ItemList, PathOrStr, 
                         DataFrame, Optional, Collection, 
                         IntsOrStrs, DataBunch, is_listy, 
                         ItemLists, TextList, SortishSampler, 
                         DataLoader, SortSampler, partial, 
                         pad_collate, TextLMDataBunch, data_collate, 
                         LanguageModelPreLoader, SPProcessor, PreProcessor )
from Bio import Seq
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import FeatureLocation, CompoundLocation
import networkx as nx

In [ ]:
#sys.path.append("../..")
from utils import ( _get_genomic_processor, get_model_clas, TextClasDataBunch, 
                    get_scores, split_data, GenomicTokenizer, get_model_LM, 
                   GenomicVocab, GenomicTextClasDataBunch, GenomicTextLMDataBunch)

In [ ]:
path = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/ecoli/')

In [ ]:
df = pd.read_csv(path/'e_coli_lm_data.csv')

In [ ]:
train_df, valid_df = split_data(df, 0.9)

In [ ]:
def remove_(text):
    return text.replace('_', '').strip()

def remove_from_list(texts):
    #print('texts are: {}'.format(texts))
    return [remove_single(text) for text in texts]

def remove_single(text):
    #print('applying ...')
    #text = text.replace('xxup ', '') # remove uppercase indicator (useless)
    text = text.upper() # make everything uppercase
    text = text.replace('XXBOS', 'xxbos') # undo uppercasing BOS
    text = text.replace('XXEOS', 'xxeos') # undo uppercasing EOS
    #text = text.replace('XXUP', 'xxup')
    #text = text.replace('XXREP', 'xxrep') # undo uppercasing REP
    return text


tmp_dir='tmp' 
cache_dir = path / 'sentencepiece_tokenizer' 
cache_dir = cache_dir / tmp_dir

gen_sp_processor = SPProcessor(sp_model=cache_dir/'spm.model', 
                               sp_vocab=cache_dir/'spm.vocab',
                               pre_rules=[remove_], 
                               post_rules=[remove_from_list], 
                               model_type='bpe',
                               max_sentence_len=20480, 
                               lang='dna', 
                               char_coverage=0.9998, 
                               tmp_dir='tmp', 
                               mark_fields=False, 
                               include_bos=True, 
                               include_eos=True,  
                               enc='utf8')

In [ ]:
class GenomicTextLMDataBunchSP(TextLMDataBunch):
    @classmethod
    def from_df(cls, path:PathOrStr, train_df:DataFrame, valid_df:DataFrame, test_df:Optional[DataFrame]=None,
                processor:PreProcessor=None, classes:Collection[str]=None, text_cols:IntsOrStrs=1,
                label_cols:IntsOrStrs=0, label_delim:str=None, bptt=70, collate_fn:Callable=data_collate, bs=64, **kwargs):
        "Create a `TextDataBunch` from DataFrames. `kwargs` are passed to the dataloader creation."
        if classes is None and is_listy(label_cols) and len(label_cols) > 1: classes = label_cols
        src = ItemLists(path, TextList.from_df(train_df, path, cols=text_cols, processor=processor),
                        TextList.from_df(valid_df, path, cols=text_cols, processor=processor))
        src = src.label_for_lm() 
        if test_df is not None: src.add_test(TextList.from_df(test_df, path, cols=text_cols))
        d1 = src.databunch(**kwargs)
        
        datasets = cls._init_ds(d1.train_ds, d1.valid_ds, d1.test_ds)            
        val_bs = bs
        datasets = [LanguageModelPreLoader(ds, shuffle=(i==0), bs=(bs if i==0 else val_bs), bptt=bptt, backwards=False) 
                    for i,ds in enumerate(datasets)]            
        dls = [DataLoader(d, b, shuffle=False) for d,b in zip(datasets, (bs,val_bs,val_bs,val_bs)) if d is not None]
        
        return cls(*dls, path=path, collate_fn=collate_fn, no_check=False)

data = GenomicTextLMDataBunchSP.from_df(path, train_df, valid_df, bs=428, processor=gen_sp_processor, text_cols=0, label_cols=1)

In [ ]:
from collections import Counter

c = Counter()

for i, dat in enumerate(data.train_dl):
    c.update(dat[0].reshape(-1).tolist())
    print('done {} of {}'.format(i, len(data.train_dl)))

In [ ]:
import matplotlib.pyplot as plt

print('{} tokens appear in DNA'.format(len(c)))
most_common = 'most common: \n'
most_common_items = c.most_common()
for i in range(10):
    print('{}: {}'.format(gen_sp_processor.vocab.textify([most_common_items[i][0]]), most_common_items[i][1]))

lens = [len(x) for x in gen_sp_processor.vocab.textify(list(c.keys())).split(' ')]
print('avg len: {:.3f}'.format(np.array(lens).mean()))
print('{} is the lowest amount of times any token appears'.format(np.array(list(c.values())).min()))
#TODO: compute average length of tokens!

plt.hist(lens, bins=7)

In [ ]:
x = list(c.values())
x.sort(reverse=True)

plt.plot(x)

print('num tokens: {}'.format(len(data.vocab.itos)))
print('first sequence: {}'.format(train_df.iloc[0].Sequence))
print('encoded: {}'.format(data.train_ds[0]))
encoding = gen_sp_processor.process_one(train_df.iloc[0].Sequence)
print('decoded: {}'.format(gen_sp_processor.vocab.textify(encoding)))

In [ ]:
data.vocab.itos[0:20]

## Train Model

In [ ]:
np.save(path/'coli_vocab_sp.npy', data.vocab.itos)

In [ ]:
config = dict(emb_sz=400, n_hid=1150, n_layers=3, pad_token=0, qrnn=False, output_p=0.25, 
                          hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15, tie_weights=True, out_bias=True)
drop_mult = 0.3

In [ ]:
learn = get_model_LM(data, drop_mult, config)

In [ ]:
learn.model

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 1e-1, moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(10, 5e-2, moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(10, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.save('coli_only_LM_sp')

In [ ]:
learn.save_encoder('coli_only_LM_enc_sp')

##  Classification downstream task

In [ ]:
path = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/ecoli/')
classification_df = pd.read_csv(path/'e_coli_promoters_dataset.csv')

In [ ]:
train_df = classification_df[classification_df.set == 'train']
valid_df = classification_df[classification_df.set == 'valid']
test_df = classification_df[classification_df.set == 'test']

In [ ]:
def remove_(text):
    return text.replace('_', '').strip()

def remove_from_list(texts):
    #print('texts are: {}'.format(texts))
    return [remove_single(text) for text in texts]

def remove_single(text):
    #print('applying ...')
    #text = text.replace('xxup ', '') # remove uppercase indicator (useless)
    text = text.upper() # make everything uppercase
    text = text.replace('XXBOS', 'xxbos') # undo uppercasing BOS
    text = text.replace('XXEOS', 'xxeos') # undo uppercasing EOS
    #text = text.replace('XXUP', 'xxup')
    #text = text.replace('XXREP', 'xxrep') # undo uppercasing REP
    return text


tmp_dir='tmp' 
cache_dir = path / 'sentencepiece_tokenizer' 
cache_dir = cache_dir / tmp_dir

gen_sp_processor = SPProcessor(sp_model=cache_dir/'spm.model', 
                               sp_vocab=cache_dir/'spm.vocab',
                               pre_rules=[remove_], 
                               post_rules=[remove_from_list], 
                               model_type='bpe',
                               max_sentence_len=20480, 
                               lang='dna', 
                               char_coverage=0.9998, 
                               tmp_dir='tmp', 
                               mark_fields=False, 
                               include_bos=True, 
                               include_eos=True,  
                               enc='utf8')

In [ ]:
class GenomicTextClasDataBunchSP(TextClasDataBunch):
    @classmethod
    def from_df(cls, path:PathOrStr, train_df:DataFrame, valid_df:DataFrame, test_df:Optional[DataFrame]=None,
                processor:PreProcessor=None, classes:Collection[str]=None, text_cols:IntsOrStrs=1,
                label_cols:IntsOrStrs=0, pad_idx=1, pad_first=True, label_delim:str=None,
                bs=64, **kwargs) -> DataBunch:
        "Create a `TextDataBunch` from DataFrames. `kwargs` are passed to the dataloader creation."
        if classes is None and is_listy(label_cols) and len(label_cols) > 1: classes = label_cols
        src = ItemLists(path, TextList.from_df(train_df, path, cols=text_cols, processor=processor),
                        TextList.from_df(valid_df, path, cols=text_cols, processor=processor))
        src = src.label_from_df(cols=label_cols, classes=classes, label_delim=label_delim)
        if test_df is not None: src.add_test(TextList.from_df(test_df, path, cols=text_cols))
        d1 = src.databunch(**kwargs)
        
        datasets = cls._init_ds(d1.train_ds, d1.valid_ds, d1.test_ds)
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=False)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs//2)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=bs, sampler=sampler, **kwargs))
            
        return cls(*dataloaders, path=path, collate_fn=collate_fn)

In [ ]:
#tok = Tokenizer(GenomicTokenizer, n_cpus=1, pre_rules=[], post_rules=[], special_cases=['xxpad'])
data_clas = GenomicTextClasDataBunchSP.from_df(path, train_df, valid_df, processor=gen_sp_processor,
                                             text_cols='Sequence', label_cols='Promoter', bs=400)

In [ ]:
clas_config = dict(emb_sz=400, n_hid=1150, n_layers=3, pad_token=0, qrnn=False, output_p=0.4, 
                       hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5)
drop_mult = 0.6

In [ ]:
learn = get_model_clas(data_clas, drop_mult, clas_config)

In [ ]:
learn.load_encoder('coli_only_LM_enc_sp')
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, 1e-1, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(3, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(3, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(5, slice(1e-4/(2.6**4),1e-4), moms=(0.8,0.7))

In [ ]:
learn.save('coli_coli_pretrain_sp', return_path=True)

In [ ]:
data_clas = GenomicTextClasDataBunchSP.from_df(path, train_df, test_df, processor=gen_sp_processor,
                                            text_cols='Sequence', label_cols='Promoter', bs=300)
learn.data = data_clas

In [ ]:
get_scores(learn)